# QAP Job Poster

This is the main code for posting open jobs for QA Professionals!

At a high-level, this notebook goes through the process. Following the code will:

* Collect jobs from different sources (ie Indeed, Google Jobs)
* Filter relevant jobs using AI
* Post those jobs to the `#jobs` channel in the QAP Slack Community!

### 1. Create the SQLite Database to save our results from collecting jobs

* import everything we'll need for this notebook
* delete the database (if exists) and create a new one

In [1]:
from typing import Dict, List

from pylenium.driver import Pylenium, PyleniumConfig

from jobs import ai, database, indeed, slack
from jobs.models import Job

database.delete()
database.create()

### 2. Use Pylenium to scrape jobs from different sources

* Currently, uses `Indeed.com`
* Eventually, will also use `Google Jobs`

In [2]:
py = Pylenium(PyleniumConfig())
scraped_jobs = indeed.scrape(py, indeed.REMOTE_QA_ENGINEER)

### 3. Save scraped jobs to our `indeed_jobs` table

We don't want to lose these results in case an error happens or if we want to use it for something else.

In [3]:
database.insert_indeed_jobs(scraped_jobs)

### 4. Use AI to filter irrelevant jobs

Unfortunately, Indeed doesn't do a great job of returning relevant results given their search and filters.

Fortunately, we can use AI to not only give us the relevant jobs, but to also rank them:

* Find jobs that would be relevant to Software QA Professionals
* Order them from junior-level to senior-level

In [4]:
# Use the least amount of tokens to get the relevant jobs from all the scraped jobs
# The AI responds with a string
ai_response: str = ai.get_relevant_jobs(scraped_jobs)

# Convert the string output from the AI to a list of dictionaries that we can use in code
ai_ranked_jobs: List[Dict] = ai.convert_relevant_jobs_to_list(ai_response)

### 5. Use the `ai_ranked_jobs` to match the Jobs in `scraped_jobs`

We can do this by creating a simple "lookup table"

In [5]:
def create_jobs_lookup(all_jobs: List[Job]) -> dict:
    """This 'lookup table' makes it easier to find relevant jobs within all jobs."""
    jobs_lookup = {}
    for job in all_jobs:
        key = f"{job.company} | {job.title}"
        if key not in jobs_lookup:
            jobs_lookup[key] = job
    return jobs_lookup

With this lookup, we can get back the full `Job` objects that match the AI's results!

In [6]:
lookup = create_jobs_lookup(scraped_jobs)
jobs = [job for j in ai_ranked_jobs if (job := lookup.get(f"{j['company']} | {j['title']}"))]

Save the jobs found by AI to our `relevant_jobs` table

In [7]:
database.insert_relevant_jobs(jobs)

### 6. Send the Top 20 Jobs to the Slack Channel

* Slack has a `Block Limit`, so we can only send the top 20

In [8]:
top_20_jobs = jobs[:20]
payload = slack.create_jobs_payload(top_20_jobs)
response = slack.post_to_channel(payload)

If the next cell runs without error, then the jobs were posted to Slack!

In [9]:
assert response.ok, f"Failed to post jobs to Slack: {response.text}"

### 7. Do the same for Google Jobs

Fortunately, Google Jobs does a much better job of returning relevant jobs, so we don't need to filter with AI

In [10]:
from jobs import google

py = Pylenium(PyleniumConfig())
google_jobs = google.scrape(py, google.SOFTWARE_QA_ENGINEER_IN_UTAH)

Save the results to the `google_jobs` table

In [11]:
database.insert_google_jobs(google_jobs)

Send the jobs to Slack

In [12]:
# Google Jobs shares 20 jobs per page, so we don't need to limit this for Slack
google_payload = slack.create_google_jobs_payload(google_jobs)
google_response = slack.post_to_channel(google_payload)

If the next cell runs without error, then the jobs were posted to Slack!

In [13]:
assert google_response.ok, f"Failed to post jobs to Slack: {google_response.text}"

## (Optional) Explore the data

In [14]:
print("Relevant Indeed Jobs Found:", len(jobs))
print("5 Sample Jobs:", "\n")
for j in jobs[:5]:
    print(j.title, j.company)

Relevant Indeed Jobs Found: 23
5 Sample Jobs: 

Junior QA Engineer Remote Technology, Inc.
Jr. QA Engineer Kyla
QA Manual Engineer EX2 Outcoding
QA Tester Bad Robot Games
QA Engineer- Mobile Actabl


In [15]:
print("Total Indeed Jobs Scraped:", len(scraped_jobs))
print("5 Sample Jobs:", "\n")
for j in scraped_jobs[:5]:
    print(j.title, j.company)

Total Indeed Jobs Scraped: 75
5 Sample Jobs: 

Angular Software Engineer ES&S Voter Registration LLC
Frontend Software Engineer Resourcely
Backend Software Engineer, Core and Monetization Pinterest
Full Stack Software Engineer, Core and Monetization Pinterest
Software Engineer Dow Jones
